In [2]:
#hiv-kode
from dash import Dash, html, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px
from prophet.plot import plot_plotly, plot_components_plotly
from datetime import datetime
from prediktiv_analyse import prediktiv_analyse
from statistikk import statistikk
import dash_leaflet as dl
import dash_leaflet.express as dlx

df = pd.read_csv(r'../data/lokasjonsdata.csv').rename(columns={'Unnamed: 0': 'Date'})

df['Date'] = pd.to_datetime(df['Date'])

app = Dash()

app.layout = html.Div(style={'fontFamily': 'Arial, sans-serif', 'padding': '20px'}, children=[
    #overskrift
    html.H1("Heftig interaktiv visualisering", style={'textAlign': 'center', 'fontSize': '36px'}),
    #diverse input
    html.Div(style={'display': 'flex', 'flexWrap': 'wrap', 'gap': '20px', 'marginBottom': '30px'}, children=[
        html.Div([
            html.Label("start-dato for data:", style={'fontSize': '18px'}),
            dcc.DatePickerSingle(
                id='Start_dato-velger',
                date=datetime.today(),
                style={'fontSize': '16px'})]),
        html.Div([
            html.Label("slutt-dato for data:", style={'fontSize': '18px'}),
            dcc.DatePickerSingle(
                id='Slutt-dato-velger',
                date=datetime.today(),
                style={'fontSize': '16px'})]),        
        html.Div([
            html.Label("Plot-type:", style={'fontSize': '18px'}),
            dcc.Dropdown(
                options=[
                    {'label': 'Visualisering', 'value': 'visualisering'},
                    {'label': 'Statistikk', 'value': 'statistikk'},
                    {'label': 'Prediktiv Analyse', 'value': 'prediktiv_analyse'}
                ],
                value='visualisering',
                id='Plot-velger',
                style={'width': '220px', 'fontSize': '16px'})
        ]),
        html.Div([
            html.Label("Velg By-data:", style={'fontSize': '18px'}),
            dcc.Dropdown(
                options=[{'label': col, 'value': col} for col in df.columns[1:]],
                value=df.columns[1] if len(df.columns) > 1 else None,
                id='By-velger',
                style={'width': '220px', 'fontSize': '16px'})
        ]),
        html.Div([
            html.Label("Juster verdi:", style={'fontSize': '18px'}),
            dcc.Slider(
                min=1, max=10, step=1, value=1,
                id='slider',
                marks={i: str(i) for i in range(1, 11)},
                tooltip={"placement": "bottom", "always_visible": True},
            )])]),
    #kart og graf
    html.Div(style={'display': 'flex', 'gap': '20px', 'marginBottom': '40px'}, children=[
    html.Div([
        html.H2("Koordinat-velger", style={'textAlign': 'center', 'fontSize': '24px'}),
        dl.Map(center=[63.4305, 10.3951], zoom=10, children=[
        dl.TileLayer(),
        ],
        id="map", style={'width': '100%', 'height': '500px'}),
        html.Div(id="click-coordinates", style={'fontSize': '16px', 'textAlign': 'center', 'marginTop': '10px'})
    ], style={'flex': '1'}),  
    html.Div([
        html.H2("Data-visualisering", style={'textAlign': 'center', 'fontSize': '24px'}),
        dcc.Graph(id='graph-content', style={'height': '500px'})
    ], style={'flex': '1'})

])

])

@callback(
    Output('graph-content', 'figure'),
    Output('click-coordinates', 'children'),
    Input('Plot-velger', 'value'),
    Input('By-velger', 'value'),
    Input('slider', 'value'),
    Input("map", "click_lat_lng"),  # Use click_lat_lng to get coordinates
)
def update_graph(plot, by, slider, click_lat_lng):
    fig = {}
    if plot == 'visualisering':
        fig = px.line(df, x='Date', y=by)
    elif plot == 'statistikk':
        fig = statistikk(by, df)
    elif plot == 'prediktiv_analyse':
        modell, forecast = prediktiv_analyse(slider, by, df[['Date', by]])
        fig = plot_plotly(modell, forecast)
    
    if click_lat_lng:  # Check if coordinates are available
        lat, lon = click_lat_lng
        coords_text = f"Klikket på Latitude: {lat:.4f}, Longitude: {lon:.4f}"
    else:
        coords_text = "Klikk på kartet for koordinater."

    return fig, coords_text

In [50]:
#kjør denne for å kjøre appen i browseren
app.run(jupyter_mode="external",port=8050)

Dash app running on http://127.0.0.1:8050/


In [3]:
#kjør denne for å kjøre appen i jupyter
app.run(jupyter_mode="inline",port=8050)